In [1]:
from transformers import AutoTokenizer

/Users/haxgeed/opt/anaconda3/envs/nlp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
tokenizer = AutoTokenizer.from_pretrained("openthaigpt/openthaigpt-1.0.0-beta-7b-chat-ckpt-hf", use_fast=False)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:
def token_n_clean(sent):
  clean = []
  tokens = tokenizer.tokenize(sent)
  for token in tokens:
    token = token.replace('▁','')
    clean.append(token)
  result_string = ' '.join(clean)
  return result_string


# **KENLM INNIT** 

In [4]:
#KENLM
!wget -O - https://kheafield.com/code/kenlm.tar.gz |tar xz
!mkdir kenlm/build

--2023-12-07 14:58:53--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491888 (480K) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 480.36K   328KB/s    in 1.5s    

2023-12-07 14:58:56 (328 KB/s) - written to stdout [491888/491888]



In [6]:
%cd kenlm/build
!cmake ..
!make -j2

/Users/haxgeed/Desktop/NGRAM/kenlm/build
CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- The C compiler identification is AppleClang 15.0.0.15000040
-- The CXX compiler identification is AppleClang 15.0.0.15000040
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/c++ - skipped
-- Detecting CXX compile features


In [7]:
!pip install https://github.com/kpu/kenlm/archive/master.zip

  Using cached https://github.com/kpu/kenlm/archive/master.zip
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# **KENLM NGRAM TRAINING**

In [8]:
!bin/lmplz -o 5 --text '/Users/haxgeed/Desktop/NGRAM/tokenized_sentences.txt' --arpa fivegram.arpa

=== 1/5 Counting and sorting n-grams ===
Reading fd 3
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 15577723 types 34580
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:414960 2:670393408 3:1256987648 4:2011180160 5:2932971264
Statistics:
1 34580 D1=0.438361 D2=0.987083 D3+=1.62454
2 3227739 D1=0.714433 D2=1.12185 D3+=1.44671
3 8701996 D1=0.874769 D2=1.2547 D3+=1.43476
4 11310772 D1=0.936161 D2=1.29411 D3+=1.3844
5 12726792 D1=0.911925 D2=1.31535 D3+=1.36603
Memory estimate for binary LM:
type     MB
probing 751 assuming -p 1.5
probing 884 assuming -r models -p 1.5
trie    357 without quantization
trie    192 assuming -q 8 -b 8 quantization 
trie    311 assuming -a 22 array pointer compression
trie    146 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sor

In [9]:
#model loading
import kenlm
model = kenlm.Model('fivegram.arpa')

Loading the LM will be faster if you build a binary file.
Reading /Users/haxgeed/Desktop/NGRAM/kenlm/build/fivegram.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


# **PERPLEXITY TESTING**

In [11]:
import math
import pandas as pd
def print_score(model, s):
  tokens = token_n_clean(s).split(" ")
  log_score = 0.0
  rows = []
  for i, (logprob, length, oov) in enumerate(model.full_scores(token_n_clean(s))):
    if i < len(tokens):
      row = {'token': tokens[i], 'probability': math.exp(logprob), 'Is OOV?': oov}
    else:
      row = {'token': 'END', 'probability': math.exp(logprob), 'Is OOV?': oov}
    rows.append(row)
    log_score += logprob
  print ('Log probability = ', log_score)
  return pd.DataFrame(rows)
def print_perplexity(model, s):

  n = len(token_n_clean(s).split())

  sum_inv_logprob = -1 * sum(score for score, _, _ in model.full_scores(token_n_clean(s)))
  perplexity = math.pow(10.0, sum_inv_logprob / n)
  return perplexity

In [14]:
print_perplexity(model, 'สมชายยิงปืนใส่คนในตลาด เสียชีวิตเป็นจำนวนมาก')

170.37483504563878

In [33]:
#prepare test set
ppl_test_path = '/Users/haxgeed/Desktop/NGRAM/testset_preplexity/testset_200k.txt'
with open(ppl_test_path, 'r', encoding='utf-8') as file:
    ppl_test_txt = file.read()

ppl_sentences = ppl_test_txt.split('\n')
ppl_sentences = [sent for sent in ppl_sentences if sent != '']

In [41]:
ppl_score_lst = []
for sent in ppl_sentences:
    score = print_perplexity(model, sent)
    ppl_score_lst.append(score)

average = sum(ppl_score_lst)/len(ppl_score_lst)
average

592.8328185504057

In [ ]:
###ทำไม test set มีแค่ 400 ปย